# Fitting Tutorial

Below is a basic exercise in fitting a quadratic polynomial to some noisy data, done using many different fitting packages to demonstrate their basic usage.

$$
\chi^2 = \sum_i^n \frac{(x_{i, \rm true} - x_{i, \rm model})^2}{\sigma_{x_i}^2}
$$

Reduced $\chi^2$:
$$
\chi^2_r = \chi^2/(n - {\rm len} \theta)
$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(0)

## The True Model (and Data Generation)

In [ ]:
def model(x, a, b, c):
    '''The model we are fitting.'''
    return a*x**2 + b*x + c

x = np.linspace(-5, 5, 256)
SIG_N = 1
prms_true = (1.1, 2.2, 3.3)
y_meas = model(x, *prms_true) + SIG_N * np.random.normal(size=x.shape)

In [ ]:
def report(prms):
    '''Generate a pretty report and plot for a set of parameters.'''
    if type(prms) == dict:
        prms = (prms['a'], prms['b'], prms['c'])
    if type(prms[0]) != float:
        prms = tuple(float(p) for p in prms)
    y = model(x, *prms)
    chisq = np.sum(np.abs(y_meas - y)**2 / SIG_N**2)
    chisq_r = chisq / (y_meas.size - len(prms))
    print(f'A:{prms[0]:7.3}, B:{prms[1]:7.3}, C:{prms[2]:7.3}')
    print(f'Chi-sq: {chisq:7.3}, Reduced Chi-sq: {chisq_r:7.3}')

    plt.figure()
    plt.plot(x, y_meas, '.', label='measured')
    plt.plot(x, y, label='model')
    plt.plot(x, y_meas - y, label='residual')
    plt.grid()
    _ = plt.legend()

In [ ]:
report(prms_true)

# Fitting

## numpy.linalg.lstsq

In [ ]:
# np.linalg.lstsq
# generic matrix-based linear chi-sq optimizer
A = np.array([x**2, x, np.ones_like(x)]).T  # design matrix
prms_opt, res2, rank, s = np.linalg.lstsq(A, y_meas, rcond=None)
report(prms_opt)

## numpy.polyfit

In [ ]:
# np.polyfit
# special case of matrix-based linear chi-sq optimizer for polynomials
prms_opt = np.polyfit(x, y_meas, deg=2)
report(prms_opt)

##  scipy.optimize.fmin

In [ ]:
from scipy.optimize import fmin

def my_chisq(prms, x, y_meas, sig_n=1):
    return np.sum(np.abs(model(x, *prms) - y_meas)**2 / sig_n**2)

# scipy.optimize.fmin
# very generic function optimization, guesses gradient from serial function evaluations
init_guess = np.array([0.1, 0.1, 0.1])
prms_opt, chisq_min, niter, ncalls, flags = fmin(my_chisq, init_guess, (x, y_meas, SIG_N), full_output=True)
report(prms_opt)

## scipy.optimize.curve_fit

In [ ]:
# scipy.optimize.curve_fit
from scipy.optimize import curve_fit

# generic function optimization via least-squares, guesses gradient from serial function evaluations
init_guess = np.array([0.1, 0.1, 0.1])
prms_opt, cov, info, msg, flag = curve_fit(model, x, y_meas, init_guess, SIG_N * np.ones_like(y_meas), full_output=True)
report(prms_opt)